In [2]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Play a Chess Game

In [3]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("PyPlot")
using PyPlot
# Pkg.add("NBInclude")
using NBInclude

`printGame(game)` prints the current position of a game.

In [4]:
function printGame(game)
    display(board(game))
end

printGame (generic function with 1 method)

Import Notebooks needed.

In [5]:
@nbinclude("Random Chess.ipynb")

playRandomMove (generic function with 1 method)

In [16]:
@nbinclude("PGN_Export.ipynb")

setGameResult (generic function with 1 method)

In [7]:
@nbinclude("ValidateUserInput.ipynb")

is_legal_move (generic function with 1 method)

In [8]:
@nbinclude("EvaluatePosition.ipynb")

terminal_evaluation (generic function with 1 method)

In [9]:
@nbinclude("Minimax.ipynb")

minimax_verbal (generic function with 1 method)

Init Cache

In [10]:
gCache = Dict()

Dict{Any, Any}()

The function `setWin(board)` takes in a `board` in a terminal position and sets the `result` value of the game and prints the result of the game as a text message.

In [11]:
function setWin(game)
    board = board(game)
    if ischeckmate(board)
        color = sidetomove(board)
        color == WHITE ? color = "BLACK" : color = "WHITE"
        setGameResult(game, color)
        print(color * " wins by checkmate")
    elseif isstalemate(board)
        print("Stalemate")
    elseif ismaterialdraw(board)
        print("Draw by insufficient material")
    elseif isrule50draw(board)
        print("Draw by 50 move draw")
    end
    setGameResult(game, "draw")
end

setWin (generic function with 1 method)

The function `playMove(game)` takes in a `game` and plays a move by the engine. Here the AI can be interchanged.

In [12]:
function playMove(game, currentscore)
    b = board(game)
    eval, move = minimax(b, currentscore, 4)
    println("Current board score: $eval")
    println("Current entries in Cache: $(length(gCache))")
    clearCache(b, move)
    domove!(game, move)
    return eval
end

playMove (generic function with 1 method)

The function `clearCache(board, move)` takes in the current `board` and the `move` that will be made next and clears the global Cache `gCache` if the move done is a pawn move or a capture. A chess position will not occure again if one of those two events happen and therefore all entries in the Cache will not be needed anymore. This will improve the runtime slightly.

In [13]:
function clearCache(board, move)
    global gCache
    if(ptype(get_piece(board, from(move))) == PAWN || ptype(get_piece(board, to(move))) != EMPTY)
        println("Cleared Cache with $(length(gCache))")
        gCache = Dict()
    end
end

clearCache (generic function with 1 method)

## Main function

The function `playGame()` allows a user to play a chess game. The AI is set via the `playMove(game)` function. After the game is finished (checkmate or draw) the game is saved as a PGN-file and saved in the /Games directory.

The function asks the user to input their name and what color they want to play as at the beginning. After that the player and the engine take alternate moves playing a chess game. 

Moves are inputted as strings which contain the pieces current location and the pieces end location. For example playing `1. e4` as white requires the string `e2e4` as input.

To resign the user can type `resign` as his move.

In [14]:
function playGame()
    # Setup Board
    game = Game()
    printGame(game)
    setGameHeaders(game)
    current_boardscore = 0  # evaluate_position(startboard())
    
    println("What's your name?")
    username = readline()
    
    println("Do you want to play as 'white' or 'black'?")
    color = readline()
    
    # Offset move if player is black
    if color != "black" && color != "white"
        print("Invalid color. Choose white or black")   
        return
    elseif color == "black"
        setheadervalue!(game, "Black", username)
        current_boardscore = playMove(game, current_boardscore)
        printGame(game)
    elseif color == "white"
        setheadervalue!(game, "White", username)
    end

    while true
        # User Move
        if !isterminal(game)
            println("Make a move. Type 'resign' to resign from the match")
            userInput = readline()
            if(userInput == "resign")
                println("You resigned the game. The engine wins.")
                break
            end
            userMove = movefromstring(userInput)
            if(userMove == nothing)
                println("Invalid User Input. Please use the format [source] [destination]. Example(e2e4)")
                continue
            end
            if(!is_legal_move(game, userMove))
                println("Invalid Chess move. Please make a other move.")
                continue
            else
                current_boardscore = evaluate_move(board(game), userMove, current_boardscore)
                clearCache(board(game), userMove)
                domove!(game, userMove)
            end
            printGame(game)
        else
            break
        end
        
        # Engine Move
        if !isterminal(game)
            # meassure time needed for the engine move
            @time begin
                current_boardscore = playMove(game, current_boardscore)
            end
            printGame(game)
        else
            break
        end
    end
    setWin(game)
    saveGameToPGN(game)
    return game
end

playGame (generic function with 1 method)

In [ ]:
playGame()